In [135]:
from sklearn.neural_network import MLPClassifier
import nltk
import string
from datasets import load_dataset
import re
from nltk.corpus import stopwords
from nltk import pos_tag
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import mlflow
from sklearnex import patch_sklearn
from warnings import filterwarnings
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [136]:
filterwarnings("ignore")

In [137]:
ag_news_dataset = load_dataset("ag_news")
stop_words = set(stopwords.words("english"))

In [153]:
#text_preprocess_types = [None, 'стемминг', 'лемматизация']
text_preprocess_types = ['лемматизация']

#words_classes = ['N', 'NJ', 'NJV', 'ALL']
words_classes = ['ALL']

#frequency_filtration_types = [None, 'low', 'high', 'both']
frequency_filtration_types = [None, 'low']

n_components = [250, 375, 500, 750]

hidden_layer_size = [375, 500, 750]

In [154]:
def base_ag_news_preprocess(text):
    tokens = text.lower()

    # Удаление спец слов
    special_words = ['reuters', 'afp', 'ap', 'usatoday.com', 'forbes.com', 'target=/stocks/quickinfo/fullquote"' ]
    for word in special_words:
        tokens = tokens.replace(word, '')
    
    pattern = r'[&lt][^<>]*&gt'
    tokens = re.sub(pattern, '', tokens)
    
    # Удаление пунктуации и цифр
    #tokens = ''.join(i if i not in set(string.punctuation) - set('-') | set(string.digits) else ' ' for i in tokens)
    tokens = ''.join(i if i not in set(string.punctuation)  | set(string.digits) else ' ' for i in tokens)
    
    # Токенизация
    tokens = nltk.word_tokenize(tokens)
    
    # Удаление стоп слов
    #stop_wordsL = stop_words - {'no','not'}
    stop_wordsL = stop_words
    tokens = [word for word in tokens if (word not in stop_wordsL and word != '-')]
    return tokens

In [155]:
def different_ag_news_preprocess(tokens, preprocess_type, words_class):
    
    # Обработка слов
    if preprocess_type == 'лемматизация':
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    elif preprocess_type == 'стемминг':
        stemmer = nltk.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Обработка частей речи
    tokens = pos_tag(tokens)
    if words_class == 'N':
        tokens = [word for word, tag in tokens if tag.startswith('N')]
    elif words_class == 'NJ':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J')]
    elif words_class == 'NJV':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J') or tag.startswith('V')]
    elif words_class == 'ALL':
        tokens = [word for word, _ in tokens]
    
    return tokens  

In [156]:
def frequency_filtration(words_dictionary, frequency_filtration_type):
    if frequency_filtration_type == 'low':
        return dict([(key, value) for key, value in words_dictionary.items() if value >= 10 ])
    elif frequency_filtration_type == 'high':
        return dict([(key, value) for key, value in words_dictionary.items() if value <= 3000])
    elif frequency_filtration_type == 'both':
        return dict([(key, value) for key, value in words_dictionary.items() if 10 <= value <= 3000])
    else:
        return words_dictionary

In [157]:
def dummy(doc):
    return doc

In [159]:
def final_ag_news_preprocess(dataset, model_type):
    # Подготовка данных
    x_train = dataset['train']['text']
    y_train = dataset['train']['label']
    
    x_test = dataset['test']['text']
    y_test = dataset['test']['label']
    
    # Базовая обработка
    for i, text in enumerate(x_train):
        x_train[i] = base_ag_news_preprocess(text)
        
    for i, text in enumerate(x_test):
        x_test[i] = base_ag_news_preprocess(text)
    
    index = 0
    # Вариативная обработка
    for preprocess_type in text_preprocess_types: 
        for words_class in words_classes:         
            words = {}
            xtr = x_train
            xte = x_test
            
            # Обработка текстов
            for i, tokens in enumerate(xtr):
                final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
                xtr[i] = final_tokens
                
                # Заполнение словаря
                for token in final_tokens:
                    if token not in words:
                        words[token] = 1
                    else:
                        words[token] += 1
            
            xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
            # Фильтрация по частоте
            for frequency_filtration_type in frequency_filtration_types: 
                filtered_words = frequency_filtration(words, frequency_filtration_type)
                token_length = len(filtered_words)
                
                # Векторизация слов
                word_list = sorted(filtered_words.keys())
                # Присвоение словам индексов
                words_indexed = {}
                for idx, word in enumerate(word_list):
                    words_indexed[word] = idx
                
                # TF-IDF
                vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
                x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
                x_test_TFIDF = vectorizer_TFIDF.transform(xte)
                
                #LSI
                for n_num in n_components:
                    SVD_TFIDF = TruncatedSVD(n_components=n_num)
                    
                    SVD_TFIDF.fit(x_train_TFIDF)
                    
                    x_train_TFIDF_SVD = SVD_TFIDF.transform(x_train_TFIDF)
                    x_test_TFIDF_SVD = SVD_TFIDF.transform(x_test_TFIDF)

                    # Построение классификаторов
                    for size in hidden_layer_size:
                        mlp_clf_TFIDF = MLPClassifier(hidden_layer_sizes=(size,), activation='relu', solver='adam', max_iter=20, random_state=42)
                    
                        # Обучение классификаторов
                        mlp_clf_TFIDF.fit(x_train_TFIDF_SVD, y_train)
                        
                        # Тестирование
                        predictions_TFIDF = mlp_clf_TFIDF.predict(x_test_TFIDF_SVD)
                        
                        # Метрики
                        macro_score_TFIDF = f1_score(y_test, predictions_TFIDF, average='macro')
                        
                        mlflow.start_run(run_name=f'{model_type}_{size}_SVD{n_num}_TFIDF_Test3')
                        mlflow.log_param('model', mlp_clf_TFIDF.__class__.__name__)
                        mlflow.log_param('preprocess_type', preprocess_type)
                        mlflow.log_param('words_class', words_class)
                        mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                        mlflow.log_param('token_length', token_length)
                        mlflow.log_param('n_components', n_num)
                        mlflow.log_param('hidden_layer_sizes', size)
                        mlflow.log_param('word vectorizer', 'TFIDF')
                        #mlflow.sklearn.log_model(clf_SVD, 'ADA(forest)')
                        mlflow.log_metric('macro_score', macro_score_TFIDF)
                        mlflow.end_run()
                        
                        index += 1
                        print(f'Итерация {index}')

In [145]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name="agNewsMLP")

2025/03/25 20:49:42 INFO mlflow.tracking.fluent: Experiment with name 'agNewsMLP' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/277054424505203466', creation_time=1742924982826, experiment_id='277054424505203466', last_update_time=1742924982826, lifecycle_stage='active', name='agNewsMLP', tags={}>

In [160]:
final_ag_news_preprocess(ag_news_dataset, 'MLP')

🏃 View run MLP_375_SVD250_TFIDF_Test3 at: http://127.0.0.1:5000/#/experiments/277054424505203466/runs/562e69c269354769b098e400f39adfc9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/277054424505203466
Итерация 1
🏃 View run MLP_500_SVD250_TFIDF_Test3 at: http://127.0.0.1:5000/#/experiments/277054424505203466/runs/50811e8238644a10b27c2a71f787fdd3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/277054424505203466
Итерация 2
🏃 View run MLP_750_SVD250_TFIDF_Test3 at: http://127.0.0.1:5000/#/experiments/277054424505203466/runs/b263704760b54eb3a49eca388a5a54da
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/277054424505203466
Итерация 3
🏃 View run MLP_375_SVD375_TFIDF_Test3 at: http://127.0.0.1:5000/#/experiments/277054424505203466/runs/523e7cf83e924540b40f1f112ad9e93b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/277054424505203466
Итерация 4
🏃 View run MLP_500_SVD375_TFIDF_Test3 at: http://127.0.0.1:5000/#/experiments/277054424505203466/runs/180f68

Ручные тесты


In [14]:
dataset = ag_news_dataset
preprocess_type = None
words_class = 'NJ'
frequency_filtration_type = None

# Подготовка данных
x_train = dataset['train']['text']
y_train = dataset['train']['label']
    
x_test = dataset['test']['text']
y_test = dataset['test']['label']
    
# Базовая обработка
for i, text in enumerate(x_train):
    x_train[i] = base_ag_news_preprocess(text)
        
for i, text in enumerate(x_test):
    x_test[i] = base_ag_news_preprocess(text)
    
words = {}
xtr = x_train
xte = x_test
            
# Обработка текстов
for i, tokens in enumerate(xtr):
    final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
    xtr[i] = final_tokens
                
    # Заполнение словаря
    for token in final_tokens:
        if token not in words:
            words[token] = 1
        else:
            words[token] += 1
            
xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
# Фильтрация по частоте
filtered_words = frequency_filtration(words, frequency_filtration_type)
token_length = len(filtered_words)
                
# Векторизация слов
word_list = sorted(filtered_words.keys())
# Присвоение словам индексов
words_indexed = {}
for idx, word in enumerate(word_list):
    words_indexed[word] = idx
    
# TF-IDF
vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
x_test_TFIDF = vectorizer_TFIDF.transform(xte)



In [123]:
SVD = TruncatedSVD(n_components=10, random_state=42)

In [124]:
SVD.fit(x_train_TFIDF)

TruncatedSVD(n_components=1500, random_state=42)

In [125]:
x_train_SVD = SVD.transform(x_train_TFIDF)
x_test_SVD = SVD.transform(x_test_TFIDF)

In [132]:
clf_SVD = MLPClassifier(hidden_layer_sizes=(250,), activation='relu', solver='adam', max_iter=30, random_state=42)
clf_SVD = clf_SVD.fit(x_train_TFIDF, y_train)

In [133]:
predictions = clf_SVD.predict(x_test_TFIDF)
macro_score = f1_score(y_test, predictions, average='macro')

In [134]:
print(macro_score)

0.8909086979348653
